# Machine Learning Poisonong Attacks in Malware Detection

## Prestart
### Import libraries

In [78]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

### Files
Store the paths for our files in variables

In [79]:
file_arm_attacker_training = "files\\arm_datasets\\arm_attacker_training.csv"
file_arm_attacker_validation = "files\\arm_datasets\\arm_attacker_validation.csv"
file_arm_attacker_test = "files\\arm_datasets\\arm_attacker_test.csv"

file_arm_victim_training = "files\\arm_datasets\\arm_victim_training.csv"
file_arm_victim_validation = "files\\arm_datasets\\arm_victim_validation.csv"
file_arm_victim_test = "files\\arm_datasets\\arm_victim_test.csv"

## Creating the arm datasets
$ 8:1:1 $ - Training:Test:Validation

### Attacker dataset
For the first model - we call it *Clean* - we will train on clean a clean dataset

#### Reading in
In this state the csv contains the filename and the label

In [80]:
arm_attacker_training = pd.read_csv(file_arm_attacker_training, header=None, index_col=False)
arm_attacker_validation = pd.read_csv(file_arm_attacker_validation, header=None, index_col=False)
arm_attacker_test = pd.read_csv(file_arm_attacker_test, header=None, index_col=False)

#### Dataset
Creating the pure dataset, only features

In [81]:
dataset_arm_attacker_training = np.asarray(arm_attacker_training.drop(columns=arm_attacker_training.columns[-2:]))
dataset_arm_attacker_validation = np.asarray(arm_attacker_validation.drop(columns=arm_attacker_validation.columns[-2:]))
dataset_arm_attacker_test = np.asarray(arm_attacker_test.drop(columns=arm_attacker_test.columns[-2:]))

#### Labels for the attacker datasets
Creating the labels for the attacker datasets
*Malware* - $ 0 $, *Benign* - $ 1 $
-**Trainining**: 800 malware, 800 benign
-**Validation**: 100 malware, 100 benign
-**Test**: 100 malware, 100 benign

In [82]:
labels_arm_attacker_training = np.asarray(arm_attacker_training[arm_attacker_training.columns[-1]])
labels_arm_attacker_validation = np.asarray(arm_attacker_validation[arm_attacker_validation.columns[-1]])
labels_arm_attacker_test = np.asarray(arm_attacker_test[arm_attacker_test.columns[-1]])

#### Filenames
Getting the filenames separated just for the fun of it

In [83]:
names_arm_attacker_training = arm_attacker_training[arm_attacker_training.columns[-2]]
names_arm_attacker_validation = arm_attacker_validation[arm_attacker_validation.columns[-2]]
names_arm_attacker_test = arm_attacker_test[arm_attacker_test.columns[-2]]

In [ ]:
names_arm_attacker_training.to_csv("files\\filenames\\arm_attacker")

### Victim dataset
For the second model - we call it *Victim* - we will train on clean a poisoned dataset

#### Reading in
In this state the csv contains the filename and the label

In [84]:
arm_victim_training = pd.read_csv(file_arm_victim_training, header=None, index_col=False)
arm_victim_validation = pd.read_csv(file_arm_victim_validation, header=None, index_col=False)
arm_victim_test = pd.read_csv(file_arm_victim_test, header=None, index_col=False)

#### Dataset
Creating the pure dataset, only features

In [85]:
dataset_arm_victim_training = np.asarray(arm_victim_training.drop(columns=arm_victim_training.columns[-2:]))
dataset_arm_victim_validation = np.asarray(arm_victim_validation.drop(columns=arm_victim_validation.columns[-2:]))
dataset_arm_victim_test = np.asarray(arm_victim_test.drop(columns=arm_victim_test.columns[-2:]))

#### Labels for the victim datasets
Creating the labels for the victim datasets
*Malware* - $ 0 $, *Benign* - $ 1 $
-**Trainining**: 800 malware, 800 benign
-**Validation**: 100 malware, 100 benign
-**Test**: 100 malware, 100 benign

In [86]:
labels_arm_victim_training = np.asarray(arm_victim_training[arm_victim_training.columns[-1]])
labels_arm_victim_validation = np.asarray(arm_victim_validation[arm_victim_validation.columns[-1]])
labels_arm_victim_test = np.asarray(arm_victim_test[arm_victim_test.columns[-1]])

#### Filenames
Getting the filenames separated just for the fun of it

In [87]:
names_arm_victim_training = arm_victim_training[arm_victim_training.columns[-2]]
names_arm_victim_validation = arm_victim_validation[arm_victim_validation.columns[-2]]
names_arm_victim_test = arm_victim_test[arm_victim_test.columns[-2]]

## Models for arm Datasets
In this section we will build the models with the exact same struture
Neural network with one *hidden layer* with a $sigmoid$ *activation function*

### Model for Attacker Data
Building, fitting and evaluating the Model with the *attacker* datasets

In [88]:
model_attacker = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
model_attacker.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()])
model_attacker.fit(dataset_arm_attacker_training, labels_arm_attacker_training, epochs=10,
                validation_data=(dataset_arm_attacker_validation, labels_arm_attacker_validation))

model_attacker.evaluate(dataset_arm_attacker_test, labels_arm_attacker_test)

Epoch 1/10
50/50 [==============================] - 0s 4ms/step - loss: 5.7462 - binary_accuracy: 0.5675 - val_loss: 3.4986 - val_binary_accuracy: 0.5700
Epoch 2/10
50/50 [==============================] - 0s 2ms/step - loss: 2.6650 - binary_accuracy: 0.5606 - val_loss: 1.8976 - val_binary_accuracy: 0.5900
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 1.4964 - binary_accuracy: 0.6231 - val_loss: 1.0399 - val_binary_accuracy: 0.6900
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.7554 - binary_accuracy: 0.7306 - val_loss: 0.5879 - val_binary_accuracy: 0.7900
Epoch 5/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4390 - binary_accuracy: 0.8075 - val_loss: 0.3977 - val_binary_accuracy: 0.8300
Epoch 6/10
50/50 [==============================] - 0s 1ms/step - loss: 0.3052 - binary_accuracy: 0.8700 - val_loss: 0.3003 - val_binary_accuracy: 0.8650
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.23

[0.1481858491897583, 0.9549999833106995]

### Model for Victim Data
Building, fitting and evaluating the Model with the *victim* datasets

In [89]:
model_victim = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
model_victim.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()])
model_victim.fit(dataset_arm_victim_training, labels_arm_victim_training, epochs=10,
                validation_data=(dataset_arm_victim_validation, labels_arm_victim_validation))

model_victim.evaluate(dataset_arm_victim_test, labels_arm_victim_test)

Epoch 1/10
50/50 [==============================] - 0s 3ms/step - loss: 1.9436 - binary_accuracy: 0.5088 - val_loss: 1.3009 - val_binary_accuracy: 0.5200
Epoch 2/10
50/50 [==============================] - 0s 1ms/step - loss: 0.8917 - binary_accuracy: 0.6269 - val_loss: 0.7221 - val_binary_accuracy: 0.6600
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 0.5439 - binary_accuracy: 0.7287 - val_loss: 0.5134 - val_binary_accuracy: 0.7200
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4114 - binary_accuracy: 0.8100 - val_loss: 0.4148 - val_binary_accuracy: 0.8050
Epoch 5/10
50/50 [==============================] - 0s 1ms/step - loss: 0.3268 - binary_accuracy: 0.8550 - val_loss: 0.3137 - val_binary_accuracy: 0.9000
Epoch 6/10
50/50 [==============================] - 0s 2ms/step - loss: 0.2671 - binary_accuracy: 0.8988 - val_loss: 0.2595 - val_binary_accuracy: 0.9100
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.21

[0.10751333832740784, 0.9750000238418579]

## POISON IT

### Files

In [90]:
file_poison_arm_BM = "files\\poison_data\\poisoned_benign_malware.csv"
file_poison_arm_MB = "files\\poison_data\\poisoned_malware_benign.csv"

### Poisoned Datasets

#### Attacker dataset

In [91]:
poisoned_arm_attacker_training = pd.read_csv(file_poison_arm_BM, index_col=False, header=None)
poisoned_arm_attacker_training = arm_attacker_training.append(poisoned_arm_attacker_training, ignore_index=True).sample(frac=1)
dataset_poisoned_arm_attacker_training = np.asarray(poisoned_arm_attacker_training.drop(columns=poisoned_arm_attacker_training.columns[-2:]))
labels_poisoned_arm_attacker_training = np.asarray(poisoned_arm_attacker_training[poisoned_arm_attacker_training.columns[-1]])

C:\Users\ZsZs\AppData\Local\Temp\ipykernel_6128\3648223893.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  poisoned_arm_attacker_training = arm_attacker_training.append(poisoned_arm_attacker_training, ignore_index=True).sample(frac=1)


### Models

#### Attacker model

In [92]:
model_attacker = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
model_attacker.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()])
model_attacker.fit(dataset_poisoned_arm_attacker_training, labels_poisoned_arm_attacker_training, epochs=10,
                validation_data=(dataset_arm_attacker_validation, labels_arm_attacker_validation))

model_attacker.evaluate(dataset_arm_attacker_test, labels_arm_attacker_test)

Epoch 1/10
51/51 [==============================] - 0s 3ms/step - loss: 0.6523 - binary_accuracy: 0.6516 - val_loss: 0.4792 - val_binary_accuracy: 0.7550
Epoch 2/10
51/51 [==============================] - 0s 2ms/step - loss: 0.4058 - binary_accuracy: 0.8422 - val_loss: 0.3847 - val_binary_accuracy: 0.8750
Epoch 3/10
51/51 [==============================] - 0s 2ms/step - loss: 0.3014 - binary_accuracy: 0.8969 - val_loss: 0.2731 - val_binary_accuracy: 0.9200
Epoch 4/10
51/51 [==============================] - 0s 2ms/step - loss: 0.2399 - binary_accuracy: 0.9174 - val_loss: 0.2294 - val_binary_accuracy: 0.9300
Epoch 5/10
51/51 [==============================] - 0s 2ms/step - loss: 0.1993 - binary_accuracy: 0.9391 - val_loss: 0.2004 - val_binary_accuracy: 0.9550
Epoch 6/10
51/51 [==============================] - 0s 2ms/step - loss: 0.1736 - binary_accuracy: 0.9497 - val_loss: 0.1885 - val_binary_accuracy: 0.9550
Epoch 7/10
51/51 [==============================] - 0s 2ms/step - loss: 0.15

[0.14977072179317474, 0.949999988079071]